
# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

## Consigna

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Intentar plantear y probar tests de analogías.
- Graficarlos.
- Obtener conclusiones.

## Resolución

### Datos
Utilizaremos como dataset "Alice's Adventures in Wonderland" (Alicia en el País de las Maravillas) de Lewis Carroll

In [ ]:
# Librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import requests
import re

import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
from gensim.models.callbacks import CallbackAny2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Leemos el archivo desde la URL usando requests
url = 'https://www.gutenberg.org/files/11/11-0.txt'
response = requests.get(url)

# Obtenemos el contenido del archivo como texto
raw_text = response.text

# Dividimos el texto en oraciones usando nltk
sentences = nltk.sent_tokenize(raw_text)

# Convertimos las oraciones en un DataFrame
df = pd.DataFrame(sentences, columns=["text"])
df.head()

,text
0,ï»¿ï»¿*** START OF THE PROJECT GUTENBERG EBOOK...
1,Down the Rabbit-Hole\r\n CHAPTER II.
2,The Pool of Tears\r\n CHAPTER III.
3,A Caucus-Race and a Long Tale\r\n CHAPTER IV.
4,The Rabbit Sends in a Little Bill\r\n CHAPTER ...


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 985


### 1 - Preprocesamiento

In [ ]:
# Función para limpiar el texto
def clean_text(text):

    # Reemplazamos saltos de línea y retornos de carro por espacios
    text = text.replace('\r', ' ').replace('\n', ' ')

    # Eliminamos caracteres no deseados (usar expresiones regulares)
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Elimina caracteres no ASCII

    return text

# Función para eliminar los metadatos de Proyecto Gutenberg
def remove_gutenberg_metadata(text):

    # Identificamos y eliminamos los metadatos iniciales (después del título del libro)
    start_idx = text.lower().find("chapter i")  # Buscar el primer capítulo de forma flexible

    # Buscamos el final del txto
    end_idx = text.lower().rfind("end of the project gutenberg")

    # Si no se encuentra el texto, devolverlo como está
    if start_idx == -1:
        start_idx = 0
    if end_idx == -1:
        end_idx = len(text)

    # Cortar el texto desde el inicio real hasta el final real
    return text[start_idx:end_idx]

# Leemos el archivo como texto completo
url = 'https://www.gutenberg.org/files/11/11-0.txt'
response = requests.get(url)
raw_text = response.text

# Limpiamos el texto y eliminamos los metadatos
cleaned_text = clean_text(raw_text)
cleaned_text = remove_gutenberg_metadata(cleaned_text)

# Dividimos en oraciones
from nltk.tokenize import sent_tokenize
sentence_tokens = sent_tokenize(cleaned_text)

# Filtramos oraciones que son solo títulos de capítulos o vacías
sentence_tokens = [sent for sent in sentence_tokens if not re.search(r'chapter \w+', sent.lower()) and len(sent.strip()) > 5]

# Convertimos las oraciones a listas de palabras usando text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

word_tokens = []
for sentence in sentence_tokens:
    word_tokens.append(text_to_word_sequence(sentence))

# Damos un vistazo a las primeras secuencias de palabras
word_tokens[:5]


[['who', 'stole', 'the', 'tarts'],
 ['down',
  'the',
  'rabbit',
  'hole',
  'alice',
  'was',
  'beginning',
  'to',
  'get',
  'very',
  'tired',
  'of',
  'sitting',
  'by',
  'her',
  'sister',
  'on',
  'the',
  'bank',
  'and',
  'of',
  'having',
  'nothing',
  'to',
  'do',
  'once',
  'or',
  'twice',
  'she',
  'had',
  'peeped',
  'into',
  'the',
  'book',
  'her',
  'sister',
  'was',
  'reading',
  'but',
  'it',
  'had',
  'no',
  'pictures',
  'or',
  'conversations',
  'in',
  'it',
  'and',
  'what',
  'is',
  'the',
  'use',
  'of',
  'a',
  'book',
  'thought',
  'alice',
  'without',
  'pictures',
  'or',
  'conversations'],
 ['so',
  'she',
  'was',
  'considering',
  'in',
  'her',
  'own',
  'mind',
  'as',
  'well',
  'as',
  'she',
  'could',
  'for',
  'the',
  'hot',
  'day',
  'made',
  'her',
  'feel',
  'very',
  'sleepy',
  'and',
  'stupid',
  'whether',
  'the',
  'pleasure',
  'of',
  'making',
  'a',
  'daisy',
  'chain',
  'would',
  'be',
  'worth

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Filtramos las oraciones para eliminar stop words
sentence_tokens_filtered = [
    [word for word in text_to_word_sequence(sentence) if word not in stop_words]
    for sentence in sentence_tokens
]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Verificamos las oraciones antes del filtrado
print("Oraciones originales (antes de filtrar palabras vacías):")
print(sentence_tokens[:5])  # Muestramos las primeras 5 oraciones originales

# Imprimimos las oraciones filtradas
print("\nOraciones filtradas (después de eliminar palabras vacías):")
print(sentence_tokens_filtered[:5])  # Muestra las primeras 5 oraciones filtradas

# Contamos la frecuencia de palabras en sentence_tokens y sentence_tokens_filtered
# Unimos todas las palabras en una sola lista
all_words_original = [word for sentence in sentence_tokens for word in text_to_word_sequence(sentence)]
all_words_filtered = [word for sentence in sentence_tokens_filtered for word in sentence]  # Ya son listas de palabras

# Contamos las frecuencias
frequency_original = Counter(all_words_original)
frequency_filtered = Counter(all_words_filtered)

# Imprimimos las 10 palabras más comunes en cada caso
print("\n10 palabras más comunes en oraciones originales:")
print(frequency_original.most_common(10))

print("\n10 palabras más comunes en oraciones filtradas:")
print(frequency_filtered.most_common(10))


Oraciones originales (antes de filtrar palabras vacías):
['Who Stole the Tarts?', 'Down the Rabbit-Hole      Alice was beginning to get very tired of sitting by her sister on the  bank, and of having nothing to do: once or twice she had peeped into  the book her sister was reading, but it had no pictures or  conversations in it, and what is the use of a book, thought Alice  without pictures or conversations?', 'So she was considering in her own mind (as well as she could, for the  hot day made her feel very sleepy and stupid), whether the pleasure of  making a daisy-chain would be worth the trouble of getting up and  picking the daisies, when suddenly a White Rabbit with pink eyes ran  close by her.', 'There was nothing so _very_ remarkable in that; nor did Alice think it  so _very_ much out of the way to hear the Rabbit say to itself, Oh  dear!', 'Oh dear!']

Oraciones filtradas (después de eliminar palabras vacías):
[['stole', 'tarts'], ['rabbit', 'hole', 'alice', 'beginning', 'get',

### 2 - Crear los vectores (word2vec)




In [ ]:
# Durante el entrenamiento la librería gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=10,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 no se usa
                     workers=2,      # si tienen más cores pueden cambiar este valor
                     sg=1,
                     alpha=0.03,    # Tasa de aprendizaje inicial
                    min_alpha=0.0001) # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtenemos el vocabulario con los tokens
w2v_model.build_vocab(word_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1576


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 687


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens_filtered,
                 total_examples=w2v_model.corpus_count,
                 epochs=120,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 150207.625
Loss after epoch 1: 122413.21875
Loss after epoch 2: 122262.1875
Loss after epoch 3: 122199.25
Loss after epoch 4: 122396.65625
Loss after epoch 5: 119458.5625
Loss after epoch 6: 119358.5
Loss after epoch 7: 118164.125
Loss after epoch 8: 113152.125
Loss after epoch 9: 109595.625
Loss after epoch 10: 108353.0
Loss after epoch 11: 107432.5
Loss after epoch 12: 104396.75
Loss after epoch 13: 102075.0
Loss after epoch 14: 102754.125
Loss after epoch 15: 101187.375
Loss after epoch 16: 101192.375
Loss after epoch 17: 99269.75
Loss after epoch 18: 96469.25
Loss after epoch 19: 92038.75
Loss after epoch 20: 92362.25
Loss after epoch 21: 92086.25
Loss after epoch 22: 92729.0
Loss after epoch 23: 92088.25
Loss after epoch 24: 90585.25
Loss after epoch 25: 91631.5
Loss after epoch 26: 90452.75
Loss after epoch 27: 89844.75
Loss after epoch 28: 91771.75
Loss after epoch 29: 92129.0
Loss after epoch 30: 91603.25
Loss after epoch 31: 91680.75
Loss after epoch 32: 91

(1034325, 1521480)

Anteriormente se hicieron diferentes pruebas en este mismo modelo, se hicieronc ambios en:


*   Se eliminaron los stop word y se volvió a entrenar el modelo con el contenido filtrado.
*   Se cambió el alpha=0.01 a 0.02 y luego a 0.03, llegando a mejores resultados.
*   Se cambió windos=5 a windows=10
*   Se aumento el número de épocas.


Conclusiones:
Se entiende que el modelo está teniendo una performance aceptable en cuanto la pérdida disminuye considerablemente en las primeras épocas, hay cierta inestabilidad en las 101 hasta la 112, pero luego se estabiliza y continua disminuyendo lo que puede sugerir que el modelo está encontrando un buen ajuste.





### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rabbit"], topn=10)

[('white', 0.8125852942466736),
 ('curiosity', 0.5300334692001343),
 ('came', 0.5210338830947876),
 ('making', 0.5150903463363647),
 ('kid', 0.5113235116004944),
 ('alice', 0.507750391960144),
 ('gloves', 0.4912770390510559),
 ('chorus', 0.48106491565704346),
 ('everything', 0.4791344404220581),
 ('loud', 0.46909743547439575)]

Este conjunto de resultados refleja que el modelo ha capturado correctamente las asociaciones semánticas clave del contexto de "Alice's Adventures in Wonderland". Tiene sentido que la palabra mas relacionada con "rabbit" sea "white" ya que uno de los personajes se llama de esa forma. A su vez, aparece la palabra "curiosity", y se puede interpretar que a "Alice" el conejo en la historia le genera curiosidad, lo cual es parte importante de la misma.

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["queen"], topn=10)

[('did', 0.3096770644187927),
 ('once', 0.26930609345436096),
 ('be', 0.2676049470901489),
 ('being', 0.2670842409133911),
 ('your', 0.2441428154706955),
 ('each', 0.2189115732908249),
 ('should', 0.21669715642929077),
 ('or', 0.1972203254699707),
 ('him', 0.1784999668598175),
 ('no', 0.16871602833271027)]

Este resultado parece indicar que el modelo ha captado que la palabra queen tiene un significado específico en el contexto del libro, y las palabras aquí mostradas son términos más generales, con menor relación directa.

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('times', 0.5937567353248596),
 ('seven', 0.5370975732803345),
 ('see', 0.5061035752296448),
 ('soup', 0.4929256737232208),
 ('cook', 0.48455601930618286),
 ('anxiously', 0.4814932346343994),
 ('tears', 0.4806995093822479),
 ('meant', 0.45831236243247986),
 ('hall', 0.4531801640987396),
 ('five', 0.4507332742214203)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["tea"], topn=5)

[('party', 0.5427762866020203),
 ('hatter', 0.5423594117164612),
 ('butter', 0.5289287567138672),
 ('dormouse', 0.4976823627948761),
 ('isnt', 0.4844900071620941)]

Este resultado muestra la capacidad del modelo para captar relaciones entre términos numéricos, pero también refleja asociaciones contextuales más amplias que pueden no ser evidentes al principio.

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["alice"])

[('few', 0.30428820848464966),
 ('have', 0.2928236126899719),
 ('off', 0.2698812484741211),
 ('all', 0.2266821414232254),
 ('my', 0.21503491699695587),
 ('then', 0.2082972377538681),
 ('than', 0.19965201616287231),
 ('but', 0.17975692451000214),
 ('your', 0.17862875759601593),
 ('not', 0.1772613376379013)]

Al ensayar con palabras que no estaban en el vocavulario, se rompe e indica que no puede encontrar esa palabra en el espacio de embeddings.

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("sister")
print(vector_love)

[ 0.72359675 -0.65630376  0.8748923  -0.6998992  -0.10982101 -0.86644155
 -0.9201054   0.16177148  0.03043167 -0.09675397 -0.42376873 -0.71386427
 -0.93904054 -0.698123   -0.6914818  -0.49163905 -0.5594138  -1.1269171
 -0.34781718 -0.31737244 -0.4012126  -0.6961447   0.14150496 -0.84146345
 -0.554353   -0.6213085   0.629245    0.19253297  0.04656784 -1.5105101
  0.71966875 -1.1712055   0.09669682 -0.65266746 -0.459713    0.29190573
 -0.15555511  0.15915354 -0.49867797 -1.06214    -0.12257899  1.706344
 -1.0654626  -0.7678665   0.2996435   0.22190249 -1.0669166  -0.3794111
 -0.11469608 -0.72338426]


In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('sister', 1.0),
 ('adventures', 0.644229531288147),
 ('dream', 0.5812686681747437),
 ('tired', 0.5325693488121033),
 ('trees', 0.5214706063270569),
 ('strange', 0.5104166865348816),
 ('late', 0.49911707639694214),
 ('writing', 0.48238879442214966),
 ('use', 0.48053568601608276),
 ('twice', 0.46622058749198914)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["queen"], topn=10)

[('knave', 0.606650710105896),
 ('king', 0.547720193862915),
 ('hurried', 0.5469918251037598),
 ('hearts', 0.5276579260826111),
 ('shrill', 0.5261293649673462),
 ('moved', 0.5155027508735657),
 ('court', 0.4903791844844818),
 ('jumped', 0.48588117957115173),
 ('reply', 0.48563677072525024),
 ('last', 0.47801950573921204)]

Este resultado refleja el contexto narrativo en el que queen está inmersa, mostrando relaciones no solo con otros personajes de la realeza, sino también con términos que describen acciones y características asociadas a ella. Esto demuestra que el modelo captura tanto relaciones semánticas explícitas (personajes, lugares) como características contextuales más sutiles.

### 5 - Test de Analogías

para llevar a cabo el test de analogías, primero vamos a explorar las palabras mas frecuentes y partir de ellas para hacer las analogías.

In [ ]:
# Obtener más palabras frecuentes (por ejemplo, las 50 más frecuentes)
most_frequent_words_extended = w2v_model.wv.index_to_key[:50]

# Filtrar las stop words
filtered_most_frequent_words_extended = [word for word in most_frequent_words_extended if word not in stop_words]

print(filtered_most_frequent_words_extended)

['said', 'alice', 'little', 'one', 'like', 'know', 'would']


In [ ]:
# Listar algunas palabras en el vocabulario
vocabulary = list(w2v_model.wv.index_to_key)
print(vocabulary[:100])  # Muestra las primeras 50 palabras

['the', 'and', 'to', 'a', 'she', 'it', 'of', 'said', 'i', 'alice', 'in', 'you', 'was', 'that', 'as', 'her', 'at', 'on', 'with', 'all', 'had', 'but', 'for', 'so', 'be', 'very', 'not', 'what', 'this', 'little', 'they', 'he', 'out', 'its', 'is', 'down', 'one', 'up', 'his', 'if', 'about', 'then', 'no', 'were', 'like', 'know', 'them', 'would', 'again', 'herself', 'went', 'do', 'have', 'when', 'could', 'or', 'there', 'thought', 'off', 'time', 'me', 'queen', 'into', 'see', 'how', 'your', 'who', 'did', 'king', 'well', 'dont', 'my', 'now', 'began', 'im', 'by', 'an', 'turtle', 'mock', 'quite', 'gryphon', 'way', 'hatter', 'are', 'think', 'their', 'just', 'much', 'some', 'go', 'say', 'thing', 'only', 'which', 'first', 'more', 'head', 'rabbit', 'here', 'voice']


In [ ]:
# Test de analogía - Caso #1

analogy_result = w2v_model.wv.most_similar(positive=['alice', 'king'], negative=['queen'], topn=1)
print("Resultado de la analogía:", analogy_result)

Resultado de la analogía: [('said', 0.5856814384460449)]


La palabra "said" arroja un número de similitud coseno de 0.58; lo que marca una relación marcada y sugiere que el modelo ha aprendido que "Alice" y "king" en el texto se relacionan con la acción de hablar o a diálogos.

In [ ]:
# Test de analogía - Caso #2

analogy_result = w2v_model.wv.most_similar(positive=['queen', 'king'], negative=['rabbit'], topn=1)
print("Resultado de la analogía:", analogy_result)

Resultado de la analogía: [('everybody', 0.5471579432487488)]


Aquí también se marca una relación entre "queen" y "king" respecto a "everybody". Lo podemos interpretar como que los primeros evocan acciones referidos a todos los personajes/púbico. Tiene sentido por la dirigencia general que se relaciona con estos 2 conceptos.

In [ ]:
# Test de analogía - Caso #3

analogy_result = w2v_model.wv.most_similar(positive=['she', 'king'], negative=['man'], topn=1)
print("Resultado de la analogía:", analogy_result)

Resultado de la analogía: [('trees', 0.38349515199661255)]


Aquí la relación es baja, este resultado podría indicar que el modelo no ha aprendido adecuadamente las relaciones de género o las relaciones específicas de personajes dentro del contexto de "Alice's Adventures in Wonderland".

In [ ]:
# Test de analogía - Caso #4

analogy_result = w2v_model.wv.most_similar(positive=['king', 'say'], negative=['queen'], topn=1)
print("Resultado de la analogía:", analogy_result)

Resultado de la analogía: [('hastily', 0.5080660581588745)]


### 6 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### 7 - Conclusiones

• El modelo demostró esr efectivo en la captura de relaciones semánticas entre palabras, como vimos en los resultados de las pruebas de analogía. La similitud coseno en los resultados sugiere que el modelo ha aprendido relaciones relevantes en el contexto de "Alice's Adventures in Wonderland".


**Resultados**:

En el primer test de analogía, la asociación de "said" con "alice" y "king" sugiere que el modelo ha internalizado interacciones significativas entre personajes. Esta relación refleja diálogos y acciones q ocurren entre ellos.
La prueba de analogía que involucra "queen" y "king" indica que el modelo ha capturado la relación de poder y autoridad entre estos personajes.
El resultado menos satisfactorio en la prueba que involucra "she", "king", y "man" podría sugerir que el modelo no ha aprendido adecuadamente las relaciones de género o que el corpus tiene limitaciones en representar estas dinámicas.


**Aspectos de Mejora:**

Se entiende que a pesar de los resultados positivos, se podrían mejorar los resultados con un corpues mas grande y diverso de esta forma mejoraría las relacion entre conceptos menos comunes.
